In [78]:
from typing import Generic, TypeVar, Optional, Tuple, Type, Any, List, Dict
from pydantic.utils import GetterDict
from pydantic.generics import GenericModel
from pydantic import (
    BaseModel, ValidationError, PydanticValueError,
    Field, constr, conint, validator, create_model
)
from xml.etree.ElementTree import fromstring
from sqlalchemy import Column, Integer, String, JSON
from sqlalchemy.dialects.postgresql import ARRAY
from sqlalchemy.ext.declarative import declarative_base

import pickle
from pathlib import Path
from datetime import datetime

In [2]:
class User(BaseModel):
    id: int
    name = "Jane Doe"

In [3]:
user = User(id='123')
user_x = User(id=123.45)

In [4]:
assert user.id == 123
assert user_x.id == 123
assert isinstance(user_x.id, int)

In [5]:
assert user.name == "Jane Doe"

In [6]:
assert user.__fields_set__ == {"id"}

In [7]:
assert user.dict() == dict(user) == {"id": 123, "name": "Jane Doe"}

In [8]:
user.id = 321
assert user.id == 321

In [9]:
class Foo(BaseModel):
    count: int
    size: Optional[float] = None

In [10]:
class Bar(BaseModel):
    apple = "x"
    banana = "y"

In [11]:
class Spam(BaseModel):
    foo: Foo
    bars: List[Bar]

In [12]:
m = Spam(foo={"count": 4}, bars=[{"apple": "x1"}, {"apple": "x2"}])
print(f"{m = }")
print(f"{m.dict() = }")

m = Spam(foo=Foo(count=4, size=None), bars=[Bar(apple='x1', banana='y'), Bar(apple='x2', banana='y')])
m.dict() = {'foo': {'count': 4, 'size': None}, 'bars': [{'apple': 'x1', 'banana': 'y'}, {'apple': 'x2', 'banana': 'y'}]}


In [13]:
Base = declarative_base()

In [14]:
class CompanyOrm(Base):
    __tablename__ = "companies"
    
    id = Column(Integer, primary_key=True, nullable=False)
    public_key = Column(String(20), index=True, nullable=False, unique=True)
    name = Column(String(63), unique=True)
    domains = Column(ARRAY(String(255)))

In [15]:
class CompanyModel(BaseModel):
    id: int
    public_key: constr(max_length=20)
    name: constr(max_length=63)
    domains: List[constr(max_length=255)]
    
    class Config:
        orm_mode = True

In [16]:
co_orm = CompanyOrm(
    id=123,
    public_key="foobar",
    name="Testing",
    domains=["example.com", "foobar.com"],
)
print(f"{co_orm = }")

co_orm = <__main__.CompanyOrm object at 0x00000215318422B0>


In [17]:
co_model = CompanyModel.from_orm(co_orm)
print(f"{co_model = }")

co_model = CompanyModel(id=123, public_key='foobar', name='Testing', domains=['example.com', 'foobar.com'])


In [18]:
class SQLModel(Base):
    __tablename__ = "my_table"
    
    id = Column("id", Integer, primary_key=True)
    metadata_ = Column("metadata", JSON)

In [19]:
class ReserveKeyModel(BaseModel):
    metadata: Dict[str, str] = Field(alias="metadata_")
    
    class Config:
        orm_mode = True

In [20]:
sql_model = SQLModel(metadata_={"key": "value"}, id=1)
pydantic_model = ReserveKeyModel.from_orm(sql_model)
print(f"{pydantic_model = }")
print(f"{pydantic_model.dict(by_alias=True) = }")

pydantic_model = ReserveKeyModel(metadata={'key': 'value'})
pydantic_model.dict(by_alias=True) = {'metadata_': {'key': 'value'}}


In [21]:
class PetClass:
    def __init__(self, *, name: str, species: str):
        self.name = name
        self.species = species

In [22]:
class PersonClass:
    def __init__(self, *, name: str, age: float = None, pets: List[PetClass]):
        self.name = name
        self.age = age
        self.pets = pets

In [23]:
class Pet(BaseModel):
    name: str
    species: str
    
    class Config:
        orm_mode = True

In [24]:
class Person(BaseModel):
    name: str
    age: float = None
    pets: List[Pet]
    
    class Config:
        orm_mode = True

In [25]:
bones = PetClass(name="Bones", species="dog")
orion = PetClass(name="Orion", species="cat")
anna = PersonClass(name="Anna", age=20, pets=[bones, orion])
anna_model = Person.from_orm(anna)
print(f"{anna_model = }")

anna_model = Person(name='Anna', age=20.0, pets=[Pet(name='Bones', species='dog'), Pet(name='Orion', species='cat')])


In [26]:
class UserGetter(GetterDict):
    def get(self, key: str, default: Any) -> Any:
        # element attributes
        if key in {"Id", "Status"}:
            return self._obj.attrib.get(key, default)
        # element children
        else:
            try:
                return self._obj.find(key).attrib["Value"]
            except (AttributeError, KeyError):
                return default

In [27]:
class UserGetterModel(BaseModel):
    Id: int
    Status: Optional[str]
    FirstName: Optional[str]
    LastName: Optional[str]
    LoggedIn: bool
    
    class Config:
        orm_mode = True
        getter_dict = UserGetter

In [28]:
xmlstring = """
<User Id="2138">
    <FirstName />
    <LoggedIn Value="true" />
</User>
"""

user = UserGetterModel.from_orm(fromstring(xmlstring))
print(f"{user = }")

user = UserGetterModel(Id=2138, Status=None, FirstName=None, LastName=None, LoggedIn=True)


In [29]:
class Location(BaseModel):
    lat = 0.1
    lng = 10.1

In [30]:
class Model(BaseModel):
    is_required: float
    gt_int: conint(gt=42)
    list_of_ints: List[int] = None
    a_float: float = None
    recursive_model: Location = None

In [31]:
data = dict(
    list_of_ints=["1", 2, "bad"],
    a_float="not a float",
    recursive_model={"lat": 4.2, "lng": "New York"},
    gt_int=21,
)

In [32]:
try:
    print(f"{Model(**data) = }")
except ValidationError as e:
    print(f"{e = }")
    print(f"{str(e) = }")
    print(f"{e.json() = }")
    print(f"{e.errors() = }")
    
    for error in e.errors():
        print(f"{error['loc'] = }, {error['type'] = }, {error['msg'] = }")
        print(f"{error['ctx'] = }") if "ctx" in error else print("{error['ctx'] = None}")

e = ValidationError(model='Model', errors=[{'loc': ('is_required',), 'msg': 'field required', 'type': 'value_error.missing'}, {'loc': ('gt_int',), 'msg': 'ensure this value is greater than 42', 'type': 'value_error.number.not_gt', 'ctx': {'limit_value': 42}}, {'loc': ('list_of_ints', 2), 'msg': 'value is not a valid integer', 'type': 'type_error.integer'}, {'loc': ('a_float',), 'msg': 'value is not a valid float', 'type': 'type_error.float'}, {'loc': ('recursive_model', 'lng'), 'msg': 'value is not a valid float', 'type': 'type_error.float'}])
str(e) = '5 validation errors for Model\nis_required\n  field required (type=value_error.missing)\ngt_int\n  ensure this value is greater than 42 (type=value_error.number.not_gt; limit_value=42)\nlist_of_ints -> 2\n  value is not a valid integer (type=type_error.integer)\na_float\n  value is not a valid float (type=type_error.float)\nrecursive_model -> lng\n  value is not a valid float (type=type_error.float)'
e.json() = '[\n  {\n    "loc": [\n  

In [33]:
class ValidatorModel(BaseModel):
    foo: str
    
    @validator("foo")
    def value_must_equal_bar(cls, v):
        if v != "bar":
            raise ValueError('value must be "bar"')
        return v

In [34]:
try:
    print(f"{ValidatorModel(foo='ber') = }")
except ValidationError as e:
    print(f"{e = }")
    print(f"{str(e) = }")
    print(f"{e.json() = }")
    print(f"{e.errors() = }")

e = ValidationError(model='ValidatorModel', errors=[{'loc': ('foo',), 'msg': 'value must be "bar"', 'type': 'value_error'}])
str(e) = '1 validation error for ValidatorModel\nfoo\n  value must be "bar" (type=value_error)'
e.json() = '[\n  {\n    "loc": [\n      "foo"\n    ],\n    "msg": "value must be \\"bar\\"",\n    "type": "value_error"\n  }\n]'
e.errors() = [{'loc': ('foo',), 'msg': 'value must be "bar"', 'type': 'value_error'}]


In [35]:
class NotABarError(PydanticValueError):
    code = "not_a_bar"
    msg_template = 'value is not "bar", got "{wrong_value}"'

In [36]:
class ValidatorModel(BaseModel):
    foo: str
    
    @validator("foo")
    def value_must_equal_bar(cls, v):
        if v != "bar":
            raise NotABarError(wrong_value=v)
        return v

In [37]:
try:
    print(f"{ValidatorModel(foo='ber') = }")
except ValidationError as e:
    print(f"{e = }")
    print(f"{str(e) = }")
    print(f"{e.json() = }")
    print(f"{e.errors() = }")

e = ValidationError(model='ValidatorModel', errors=[{'loc': ('foo',), 'msg': 'value is not "bar", got "ber"', 'type': 'value_error.not_a_bar', 'ctx': {'wrong_value': 'ber'}}])
str(e) = '1 validation error for ValidatorModel\nfoo\n  value is not "bar", got "ber" (type=value_error.not_a_bar; wrong_value=ber)'
e.json() = '[\n  {\n    "loc": [\n      "foo"\n    ],\n    "msg": "value is not \\"bar\\", got \\"ber\\"",\n    "type": "value_error.not_a_bar",\n    "ctx": {\n      "wrong_value": "ber"\n    }\n  }\n]'
e.errors() = [{'loc': ('foo',), 'msg': 'value is not "bar", got "ber"', 'type': 'value_error.not_a_bar', 'ctx': {'wrong_value': 'ber'}}]


In [38]:
class UserLoginRecord(BaseModel):
    id: int
    name = "John Doe"
    last_login: datetime = None

In [39]:
m = UserLoginRecord.parse_obj({"id": 123, "name": "James"})

In [40]:
print(f"{m = }")

m = UserLoginRecord(id=123, last_login=None, name='James')


In [41]:
try:
    print(f"{UserLoginRecord.parse_obj(['not', 'a', 'dict']) = }")
except ValidationError as e:
    print(f"{e = }")
    print(f"{str(e) = }")
    print(f"{e.json() = }")
    print(f"{e.errors() = }")

e = ValidationError(model='UserLoginRecord', errors=[{'loc': ('__root__',), 'msg': 'UserLoginRecord expected dict not list', 'type': 'type_error'}])
str(e) = '1 validation error for UserLoginRecord\n__root__\n  UserLoginRecord expected dict not list (type=type_error)'
e.json() = '[\n  {\n    "loc": [\n      "__root__"\n    ],\n    "msg": "UserLoginRecord expected dict not list",\n    "type": "type_error"\n  }\n]'
e.errors() = [{'loc': ('__root__',), 'msg': 'UserLoginRecord expected dict not list', 'type': 'type_error'}]


In [42]:
m = UserLoginRecord.parse_raw('{"id": 123, "name": "James"}')
print(f"{m = }")

m = UserLoginRecord(id=123, last_login=None, name='James')


In [43]:
pickle_data = pickle.dumps({
    'id': 123,
    'name': 'James',
    'last_login': datetime(2017, 7, 14)
})
m = UserLoginRecord.parse_raw(
    pickle_data, content_type="application/pickle", allow_pickle=True
)
print(f"{m = }")

m = UserLoginRecord(id=123, last_login=datetime.datetime(2017, 7, 14, 0, 0), name='James')


In [44]:
path = Path("data.json")
path.write_text('{"id": 123, "name": "James"}')
m = UserLoginRecord.parse_file(path)
print(f"{m = }")

m = UserLoginRecord(id=123, last_login=None, name='James')


In [45]:
class TrustedUser(BaseModel):
    id: int
    age: int
    name: str = "John Doe"

In [46]:
original_user = TrustedUser(id=123, age=32)
user_data = original_user.dict()
print(f"{user_data = }")

fields_set = original_user.__fields_set__
print(f"{fields_set = }")

user_data = {'id': 123, 'age': 32, 'name': 'John Doe'}
fields_set = {'age', 'id'}


In [47]:
new_user = TrustedUser.construct(_fields_set=fields_set, **user_data)
print(repr(new_user))
print("{new_user.__fields_set__ = }")

TrustedUser(id=123, age=32, name='John Doe')
{new_user.__fields_set__ = }


In [48]:
bad_user = TrustedUser.construct(id="dog")
print(repr(bad_user))

TrustedUser(id='dog', name='John Doe')


In [49]:
DataT = TypeVar("DataT")

In [50]:
class Error(BaseModel):
    code: int
    message: str

In [51]:
class DataModel(BaseModel):
    numbers: List[int]
    people: List[str]

In [52]:
class Response(GenericModel, Generic[DataT]):
    data: Optional[DataT]
    error: Optional[Error]
    
    @validator("error", always=True)
    def check_consistency(cls, v, values):
        if v is not None and values["data"] is not None:
            raise ValueError("must not provide both data and error")
        if v is None and values.get("data") is None:
            raise ValueError("must provide data or error")
        return v

In [53]:
data = DataModel(numbers=[1, 2, 3], people=[])
error = Error(code=404, message="not found")

In [54]:
print(f"{Response[int](data=1) = }")
print(f"{Response[str](data='value') = }")
print(f"{Response[str](data='value').dict() = }")
print(f"{Response[DataModel](data=data).dict() = }")

Response[int](data=1) = Response[int](data=1, error=None)
Response[str](data='value') = Response[str](data='value', error=None)
Response[str](data='value').dict() = {'data': 'value', 'error': None}
Response[DataModel](data=data).dict() = {'data': {'numbers': [1, 2, 3], 'people': []}, 'error': None}


In [55]:
print(f"{Response[Error](error=error).dict() = }")

Response[Error](error=error).dict() = {'data': None, 'error': {'code': 404, 'message': 'not found'}}


In [56]:
try:
    Response[int](data="value")
except ValidationError as e:
    print(e)

2 validation errors for Response[int]
data
  value is not a valid integer (type=type_error.integer)
error
  must provide data or error (type=value_error)


In [57]:
TypeX = TypeVar("TypeX")

In [58]:
class BaseClassType(GenericModel, Generic[TypeX]):
    x: TypeX

In [59]:
class ChildClassType(BaseClassType[TypeX], Generic[TypeX]):
    pass

In [60]:
print(ChildClassType[int](x=1))

x=1


In [61]:
TypeX = TypeVar("TypeX")
TypeY = TypeVar("TypeY")
TypeZ = TypeVar("TypeZ")

In [62]:
class BaseRootClass(GenericModel, Generic[TypeX, TypeY]):
    x: TypeX
    y: TypeY

In [63]:
class ChildModel(BaseRootClass[int, TypeY], Generic[TypeY, TypeZ]):
    z: TypeZ

In [64]:
print(ChildModel[str, int](x=1, y="y", z=3))

x=1 y='y' z=3


In [65]:
class ResponseConcrete(GenericModel, Generic[DataT]):
    data: DataT
    
    @classmethod
    def __concrete_name__(cls: Type[Any], params: Tuple[Type[Any], ...]) -> str:
        return f"{params[0].__name__.title()}Response"

In [66]:
print(repr(ResponseConcrete[int](data=1)))
print(repr(ResponseConcrete[str](data='a')))

IntResponse(data=1)
StrResponse(data='a')


In [67]:
T = TypeVar("T")

In [68]:
class InnerT(GenericModel, Generic[T]):
    inner: T

In [69]:
class OuterT(GenericModel, Generic[T]):
    outer: T
    nested: InnerT[T]

In [70]:
nested = InnerT[int](inner=1)
print(OuterT[int](outer=1, nested=nested))

outer=1 nested=InnerT[int](inner=1)


In [71]:
try:
    nested = InnerT[str](inner="a")
    print(OuterT[int](outer="a", nested=nested))
except ValidationError as e:
    print(e)

2 validation errors for OuterT[int]
outer
  value is not a valid integer (type=type_error.integer)
nested -> inner
  value is not a valid integer (type=type_error.integer)


In [72]:
AT = TypeVar("AT")
BT = TypeVar("BT")

In [73]:
class RootModel(GenericModel, Generic[AT, BT]):
    a: AT
    b: BT

In [74]:
print(RootModel(a="a", b="b"))

a='a' b='b'


In [75]:
IntT = TypeVar("IntT", bound=int)
typevar_model = RootModel[int, IntT]
print(typevar_model(a=1, b=1))

a=1 b=1


In [76]:
try:
    print(typevar_model(a="a", b="a"))
except ValidationError as e:
    print(e)

2 validation errors for RootModel[int, IntT]
a
  value is not a valid integer (type=type_error.integer)
b
  value is not a valid integer (type=type_error.integer)


In [77]:
concrete_model = typevar_model[int]
print(concrete_model(a=1, b=1))

a=1 b=1


In [79]:
DynamicFooBar = create_model("DynamicFooBarModel", foo=(str, ...), bar=123)

In [80]:
class StaticFooBarModel(BaseModel):
    foo: str
    bar: int = 123

In [81]:
class FooModel(BaseModel):
    foo: str
    bar: int = 123

In [82]:
BarModel = create_model(
    "BarModel",
    apple="russet",
    banana="yellow",
    __base__=FooModel,
)

In [83]:
print(f"{BarModel = }")
print(f"{BarModel.__fields__.keys() = }")

BarModel = <class 'pydantic.main.BarModel'>
BarModel.__fields__.keys() = dict_keys(['foo', 'bar', 'apple', 'banana'])


In [84]:
def username_alphanumeric(cls, v):
    assert v.isalnum(), "must be alphanumeric"
    return v

In [85]:
validators = {"username_validator": validator("username")(username_alphanumeric)}

In [86]:
DynamicUserModel = create_model(
    "DynamicUserModel",
    username=(str, ...),
    __validators__=validators,
)

In [87]:
user = DynamicUserModel(username="abdurrakib")
print(f"{user = }")

user = DynamicUserModel(username='abdurrakib')


In [88]:
try:
    print(f"{DynamicUserModel(username='rifat%1508') = }")
except ValidationError as e:
    print(str(e))

1 validation error for DynamicUserModel
username
  must be alphanumeric (type=assertion_error)
